In [1]:
from database.sec import SEC
from database.market import Market
from transformer.sec_transformer import SECTransformer
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm

In [2]:
sec = SEC()
market = Market()

In [ ]:
sec.connect()
sec.create_index("filings","cik")
sec.create_index("filings","adsh")
for year in tqdm(range(2012,2024)):
    for quarter in range(1,5):
        try:
            sub = SECTransformer.transform(year,quarter,"sub")[["adsh","cik","filed"]]
            num = SECTransformer.transform(year,quarter,"num")
            for adsh in num["adsh"].unique():
                try:
                    filing = num[num["adsh"]==adsh].pivot_table(index="adsh",columns="tag",values="value").reset_index()
                    for column in filing.columns:
                        if "." in column:
                            new = column.replace(".","-")
                            filing.rename(columns={column:new},inplace=True)
                    complete = p.column_date_processing(filing)
                    complete = complete.merge(sub,on="adsh",how="left")
                    sec.store("filings",complete)
                except Exception as e:
                    print(adsh,str(e))
        except Exception as e:
            print(year,quarter,str(e))
sec.disconnect()

  0%|                                                                                                                                                                                   | 0/12 [00:00<?, ?it/s]E:\projects\longshot_venv\longshot\database\adatabase.py:40: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  records = data.to_dict("records")
